In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
# %matplotlib inline
pd.set_option('precision', np.float32)

import torch
from torch import nn
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
vTest = 1000

OptionError: Pattern matched multiple keys

Claude: 根据提供的notebook,Claude2对代码的建议和注释如下:
1. 数据处理部分可以添加更多注释,说明读取的数据字段含义,以及对数据的检查和处理思路。
2. 模型定义部分可以添加注释,说明输入和输出维度的设定依据,网络层数和神经元数量的选择依据等。
3. 训练评估部分的注释可以更详细,说明采用的数据分割方式,训练过程中的超参数,以及评估指标的计算方法。
4. 模型训练结果可以进行更深入的分析,检查是否存在过拟合或欠拟合,并结合业务目标提出后续优化思路。 
5. 可以添加模型效果可视化的代码,如Loss曲线,预测结果示例等,更直观地呈现模型效果。
6. 添加模型保存和加载的代码,便于训练后的模型重用。
7. 可以尝试不同模型结构或训练策略的比较实验。
8. 可以基于代码封装出函数或类,提高代码的复用性和模块化程度。
9. 添加类型注释,可以提高代码可读性和健壮性。
总体而言,该notebook内容完整,但仍有进一步提升注释、可视化、模块化、模型分析等方面的优化空间。

In [2]:
sql="SELECT * FROM ds_qf60"
# sql="SELECT * FROM ds_qf60_2"
df = pd.read_sql_query(sql, conn, index_col=None)
del df['index']

In [3]:
df['class0'] = df['test10_10']
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294721 entries, 0 to 294720
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   pct_change    294721 non-null  float64
 1   diff          294721 non-null  float64
 2   bar           294721 non-null  float64
 3   jx_days_ud60  294721 non-null  int64  
 4   jx_xl_250     294721 non-null  float64
 5   jx_xl_120     294721 non-null  float64
 6   jx_xl_60      294721 non-null  float64
 7   jx_xl_20      294721 non-null  float64
 8   jx_xl_10      294721 non-null  float64
 9   jx_xl_5       294721 non-null  float64
 10  jx_zs_5       294721 non-null  int64  
 11  jx_zs_10      294721 non-null  int64  
 12  jx_zs_20      294721 non-null  int64  
 13  jx_zs_60      294721 non-null  int64  
 14  lj_fl_1_3     294721 non-null  float64
 15  lj_fl_3_10    294721 non-null  float64
 16  lj_fl_5_20    294721 non-null  float64
 17  lj_fl_20_60   294721 non-null  float64
 18  test

(None, (294721, 22))

In [4]:
df.class0.unique(), df.class0.value_counts()

(array([0, 1], dtype=int64),
 0    257233
 1     37488
 Name: class0, dtype: int64)

In [5]:
Y_data = df.class0.values.reshape(-1,1)  # 转换成 pd type array
# Y_data
# Y = torch.from_numpy(Y_data).type(torch.float32)  # 转换成tensor
Y = torch.from_numpy(Y_data).type(torch.FloatTensor)
Y.shape, Y.size(0)

(torch.Size([294721, 1]), 294721)

In [6]:
X_columns = ['pct_change', 'diff', 'bar', 'jx_days_ud60', 'jx_xl_250', 'jx_xl_120',
       'jx_xl_60', 'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 'jx_zs_5', 'jx_zs_10',
       'jx_zs_20', 'jx_zs_60', 'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20',
       'lj_fl_20_60']
# X_columns
X_data = df[X_columns]  
X_data.shape, X_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294721 entries, 0 to 294720
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   pct_change    294721 non-null  float64
 1   diff          294721 non-null  float64
 2   bar           294721 non-null  float64
 3   jx_days_ud60  294721 non-null  int64  
 4   jx_xl_250     294721 non-null  float64
 5   jx_xl_120     294721 non-null  float64
 6   jx_xl_60      294721 non-null  float64
 7   jx_xl_20      294721 non-null  float64
 8   jx_xl_10      294721 non-null  float64
 9   jx_xl_5       294721 non-null  float64
 10  jx_zs_5       294721 non-null  int64  
 11  jx_zs_10      294721 non-null  int64  
 12  jx_zs_20      294721 non-null  int64  
 13  jx_zs_60      294721 non-null  int64  
 14  lj_fl_1_3     294721 non-null  float64
 15  lj_fl_3_10    294721 non-null  float64
 16  lj_fl_5_20    294721 non-null  float64
 17  lj_fl_20_60   294721 non-null  float64
dtypes: f

((294721, 18), None)

In [7]:
# X = torch.from_numpy(X_data.values).type(torch.FloatTensor)
X = torch.from_numpy(X_data.values).type(torch.float32)
X.shape, X.size()

(torch.Size([294721, 18]), torch.Size([294721, 18]))

In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

class Model4(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 32)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(32, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(29, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 64)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [9]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# HR_dataset = TensorDataset(X, Y)
# test_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

In [10]:
# Ver 4.2 - sklearn mode
lr = 0.00015
def get_model():
    model = Model4()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

model, optim = get_model()
model, optim 

loss_fn = nn.BCELoss()
# batch = 64
batch = 1024
# batch = 8192
batch = 16384
# batch = 32768
epochs = 10
no_of_batches = len(X)//batch
from torch.utils.data import TensorDataset, DataLoader

def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)

train_x = torch.from_numpy(train_x.values).type(torch.float32)
train_y = torch.from_numpy(train_y).type(torch.float32)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32)
test_y = torch.from_numpy(test_y).type(torch.float32)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

train_x.shape, test_x.shape, train_y.shape, test_y.shape

(torch.Size([221040, 18]),
 torch.Size([73681, 18]),
 torch.Size([221040, 1]),
 torch.Size([73681, 1]))

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [11]:
%%time
# epochs = 10
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:  0  train_loss:  0.5699  Acc： 0.8721 0  test_loss:  0.5689  Acc： 0.8748
epoch:  1  train_loss:  0.437  Acc： 0.8721 1  test_loss:  0.4342  Acc： 0.8748
epoch:  2  train_loss:  0.3969  Acc： 0.8721 2  test_loss:  0.3926  Acc： 0.8748
epoch:  3  train_loss:  0.3929  Acc： 0.8721 3  test_loss:  0.3877  Acc： 0.8748
epoch:  4  train_loss:  0.3926  Acc： 0.8721 4  test_loss:  0.3874  Acc： 0.8748
epoch:  5  train_loss:  0.3923  Acc： 0.8721 5  test_loss:  0.3872  Acc： 0.8748
epoch:  6  train_loss:  0.3919  Acc： 0.8721 6  test_loss:  0.3869  Acc： 0.8748
epoch:  7  train_loss:  0.3917  Acc： 0.8721 7  test_loss:  0.3867  Acc： 0.8748
epoch:  8  train_loss:  0.3915  Acc： 0.8721 8  test_loss:  0.3864  Acc： 0.8748
epoch:  9  train_loss:  0.3912  Acc： 0.8721 9  test_loss:  0.3862  Acc： 0.8748
CPU times: total: 26.3 s
Wall time: 39.3 s


In [12]:
# 最后一次计算结果：

epoch_accuracy = accuracy(model(train_x), train_y)
epoch_loss = loss_fn(model(train_x), train_y).data.item()
epoch_test_accuracy = accuracy(model(test_x), test_y)
epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
print('epoch: ', 
    ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
    ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:   train_loss:  0.3912  Acc： 0.8721  test_loss:  0.3862  Acc： 0.8748


In [13]:
# print( loss_fn(model(train_x), train_y).data.item() )
# accuracyRate = ((model(train_x).data.numpy()).astype('int') == train_y.numpy().astype('int')).mean()
# print("训练数据 - 准确率%：", accuracyRate*100) 

# print( loss_fn(model(test_x), test_y).data.item() )
# accuracyRate = ((model(test_x).data.numpy()).astype('int') == test_y.numpy().astype('int')).mean()
# print("测试数据 - 准确率%：", accuracyRate*100) 

数据准确性验证
过拟合/欠拟合
- 过拟合：对于已知数据过度拟合，对未知数据预测性差
- 欠拟合：对于已知数据拟合不够，对未知数据预测性差
  
tensor合并
- 

In [14]:
train_x.shape, train_y.shape

(torch.Size([221040, 18]), torch.Size([221040, 1]))

In [15]:
train_cat = torch.cat([train_x, train_y],dim=1)   # 沿着Dimension 1进行合并

In [16]:
train_cat.shape

torch.Size([221040, 19])

In [17]:
# train_all.numpy()
np.savetxt('../dataset/test20.csv', train_cat.numpy(),fmt='%.8f',delimiter=',') # 直接覆盖

In [18]:
train_np = np.loadtxt('../dataset/test20.csv', dtype = np.float32, delimiter=',')

In [19]:
train_np.shape, train_np

((221040, 19),
 array([[-1.5533   ,  0.0845455,  0.0594598, ...,  1.13678  ,  0.94658  ,
          1.       ],
        [-1.1204   , -0.182787 , -0.55973  , ...,  1.26599  ,  0.853989 ,
          0.       ],
        [-1.77     ,  0.0553558,  0.13469  , ...,  0.99355  ,  0.736688 ,
          0.       ],
        ...,
        [-1.83     , -0.317725 , -0.0152966, ...,  1.24075  ,  0.77284  ,
          0.       ],
        [ 1.9833   , -0.0489194,  0.147776 , ...,  1.03801  ,  0.963012 ,
          0.       ],
        [ 2.15     , -0.0879402,  0.0843983, ...,  1.55278  ,  0.761674 ,
          0.       ]], dtype=float32))